## Description

Produce a plot showing what fraction of the genome is covered at different depths.

In [1]:
import phase3_data

In [2]:
v3 = phase3_data.release_data()

In [3]:
import numba
import numpy as np
import dask.array as da

In [4]:
# kubernetes cluster setup
n_workers = 5
from dask_kubernetes import KubeCluster
cluster = KubeCluster()
cluster.scale_up(n_workers)
#cluster.adapt(minimum=1, maximum=n_workers)
cluster

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:  tcp://10.32.38.184:43559
distributed.scheduler - INFO -   dashboard at:                     :8787


In [5]:
# dask client setup
from dask.distributed import Client, progress
client = Client(cluster)
client

distributed.scheduler - INFO - Receive client connection: Client-5e00297a-c1f9-11ea-86cd-aeaed04a9f5f
distributed.core - INFO - Starting established connection


Client Scheduler: tcp://10.32.38.184:43559 Dashboard: /user/nicholasharding/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
all_meta = v3.load_metadata_by_sampleset(v3.all_samplesets)

distributed.scheduler - INFO - Register tcp://10.33.64.21:40841
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.33.64.21:40841
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.33.136.4:34003
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.33.136.4:34003
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.35.63.28:40689
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.35.63.28:40689
distributed.core - INFO - Starting established connection
/opt/conda/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [7]:
chromosomes = "2L", "2R", "3L", "3R", "X"

In [8]:
males_select = all_meta.sex_call == "M"

select = (all_meta.species_gambcolu_arabiensis == "gamb_colu") & males_select

In [9]:
gt = v3.load_calldata_by_sampleset("X", v3.all_samplesets)

In [10]:
gt_sel = da.compress(select.values, gt, axis=1)

In [11]:
gt_sel.shape

(23385349, 220, 2)

In [12]:
import pandas as pd

In [13]:
pd.crosstab(all_meta.sampleset, all_meta.sex_call)

sex_call,F,M,UKN
sampleset,,,
AG1000G-AO,77,4,0
AG1000G-BF-A,151,30,0
AG1000G-BF-B,74,28,0
AG1000G-BF-C,13,0,0
AG1000G-CD,44,32,0
AG1000G-CF,73,0,0
AG1000G-CI,80,0,0
AG1000G-CM-A,258,45,0
AG1000G-CM-B,97,0,0


In [14]:
import allel

In [15]:
het = allel.GenotypeDaskArray(gt_sel).is_het()

In [16]:
gamb_colu_mask = v3.load_mask("X", "gamb_colu")

In [17]:
het_filtered = da.compress(gamb_colu_mask, het, axis=0)

In [18]:
is_called = allel.GenotypeDaskArray(gt_sel).is_called()

In [19]:
is_called[:10].compute()

distributed.scheduler - INFO - Remove worker tcp://10.33.136.4:34003
distributed.core - INFO - Removing comms to tcp://10.33.136.4:34003
distributed.scheduler - INFO - Register tcp://10.33.136.4:34003
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.33.136.4:34003
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Remove worker tcp://10.33.64.21:40841
distributed.core - INFO - Removing comms to tcp://10.33.64.21:40841
distributed.scheduler - INFO - Register tcp://10.33.64.21:40841
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.33.64.21:40841
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Remove worker tcp://10.33.136.4:34003
distributed.core - INFO - Removing comms to tcp://10.33.136.4:34003
distributed.scheduler - INFO - Register tcp://10.33.136.4:34003
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.33.136.4:34003
distributed.core - I

KilledWorker: ('array-original-17d3711d3c1665ee306c18e2da88a997', <Worker 'tcp://10.33.64.21:40841', memory: 0, processing: 1>)

In [19]:
is_called_sum_all = is_called.sum(axis=0).compute()

distributed.scheduler - INFO - Remove worker tcp://10.33.124.5:42325
distributed.core - INFO - Removing comms to tcp://10.33.124.5:42325
distributed.scheduler - INFO - Register tcp://10.33.124.5:42325
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.33.124.5:42325
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Remove worker tcp://10.33.136.3:35763
distributed.core - INFO - Removing comms to tcp://10.33.136.3:35763
distributed.scheduler - INFO - Register tcp://10.33.136.3:35763
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.33.136.3:35763
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Remove worker tcp://10.33.124.5:42325
distributed.core - INFO - Removing comms to tcp://10.33.124.5:42325
distributed.scheduler - INFO - Register tcp://10.33.124.5:42325
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.33.124.5:42325
distributed.core - I

KilledWorker: ('array-original-a59b11ea799336690ee3102aa37282d7', <Worker 'tcp://10.33.136.3:35763', memory: 0, processing: 1>)

In [ ]:
is_called_sum_filtered = da.compress(gamb_colu_mask, is_called, axis=0).sum(axis=0).compute()